<img src='logo.png' style='display: block;height: 61px;float: left;padding: .75rem 1.25rem;'>

# Desafío - Boosting Classifiers
* Para realizar este desafío debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el `.zip` a la seccióncorrespondiente.

## Descripción

Para esta sesión trabajaremos con una base de datos sobre rotación de clientes en una compañia de telecomunicaciones. El archivo contiene 3333 registros y 20 atributos. El vector objetivo a modelar es la tasa de rotación entre los clientes de una compañia de telecomunicaciones churn . Los atributos existentes hacen referencia a características de la cuenta de cada cliente.


Lista de atributos:
* `State`: Estado de Estados Unidos
* `Account Length`: Tiempo en que la cuenta ha sido activada
* `Area Code`: código de área.
* `international plan`: Plan internacional activiado
* `voice mail plan`: Plan de mensajes de voz activado
* `number_vmail_messages`: Cantidad de mensajes de voz.
* `total_day_minutes`: Cantidad de minutos ocupados en la mañana. 
* `total_day_calls`: Cantidad de llamadas realizadas en la mañana. 
* `total_day_charge`: Cobros realizados en la mañana.
* `total_eve_minutes`: Cantidad de minutos ocupados en la tarde.
* `total_eve_calls`: Cantidad de llamadas realizadas en la tarde. 
* `total_eve_charge`: Cobros realizados en la tarde.
* `total_night_calls`: Cantidad de llamadas realizadas en la noche. 
* `tota_night_minutes`: Cantidad de minutos ocupados en la noche.
* `total_night_charge`: Cobros realizados en la noche.
* `total_intl_minutes`: Cantidad de minutos ocupados en llamadas internacionales. 
* `total_intl_calls`: Cantidad de llamadas internacionales realizadas. 
* `total_intl_charge`: Cobros realizados por llamadas internacionales.
* `churn`: 1 si el cliente se cambió de compañia, 0 de los contrario.

Los datos provienen del paquete `AppliedPreditiveModeling` de `R`.

## Ejercicio 1: Preprocesamiento

* Grafique el comportamiento distributivo de los atributos y de la variable dependiente. Reporte brevemente el comportamiento de las variables.
* En base al comportamiento de los atributos, considere si es necesario implementar alguna recodificación o transformación de atributo. Algunas normas a seguir:
  * Para las variables categóricas, recodifíquelas en variables binarias.
  * Para aquellas variables numéricas que presenten alto sesgo, pueden transformarlas con su logaritmo.

> Se utilizan librerías bases para el tratamiento de datos y algunos aspectos básicos de cálculo y gráficos.

In [1]:
# Se importa libreria para el manejo de bases de datos
import pandas as pd
# Se importa libreria para el manejo de operaciones de cálculo
import numpy as np
# Se importa libreria para el manejo de gráficos
import matplotlib.pyplot as plt
# Se importa libreria para manejo de funciones estadisticas y probabilidades
import seaborn as sns
# Librería para visualizar patrones de datos perdidos
import missingno as msngo
# Se importa libreria para el menejo de warning
import warnings

> Definimos algunos aspectos de ambiente y valores por defecto de visualización.

In [2]:
# Por defecto, matplotlib crea una figura en una ventana separada.
# podemos hacer que las figuras aparezcan en línea dentro del notebook; lo hacemos ejecutando:
%matplotlib inline
# Se ignoran los warning para evitar suciedad en la ejecución
warnings.filterwarnings(action='ignore')
# Se define el estilo de gráficos a usar
plt.style.use('seaborn-pastel')
# Se define el tamaño de los paños de los gráficos por defecto
plt.rcParams['figure.figsize'] = (14, 8)
# Dado que vamos a supervisar datos no limitaremos la cantidad de columnas a mostrar en el despliegue del dataframe
pd.options.display.max_columns = None

> Se utiliza <strong>librería propia</strong> que continen funciones auxiliares, necesarias para el desarrollo de desafíos.

In [3]:
# Se importa libreria con funciones auxiliares
import ancilliary_funcs as afx

> Se utilizan librerías <strong>sklearn</strong> para el modelo, el tratamiento de escala de valores de los atributos y seleccion de set de datos:

In [4]:
# Método para escalar datos
from sklearn.preprocessing import StandardScaler
# Método para la selección de datos entre entrenamiento, test y CV
from sklearn.model_selection import train_test_split, GridSearchCV
# Método para reportar las métricas de modelos de clasificación
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
# Método para generar un modelo de regresión con arboles de decisión
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## Ejercicio 2: Comparación de AdaBoost y Gradient Boosting

* Entrene los clasificadores AdaBoost y Gradient Boosting para mejorar su capacidad predictiva en la medida de lo posible. Para ello, implemente una búsqueda de grilla con las siguientes especificaciones:


|__Modelo__   | __Grilla__  |
|---|---| 
| AdaBoostClassifier| {'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [50, 100, 500, 1000, 2000]}|
| GradientBoostingClassifier |  {'learning_rate': [0.01, 0.1, 0.5],'n_estimators': [50, 100, 500, 1000, 2000], 'subsample': [0.1,0.5,0.9]}|

* Si el tiempo de computación es alto, puede implementar la búsqueda con 1 validación cruzada.
* Reporte las métricas para los mejores modelos.

## Ejercicio 3: Principales factores asociados

* Con el mejor modelo, reporte la importancia relativa de los atributos y comente cuáles son los que aumentan la probabilidad de fuga en los clientes.
* tip: Pueden implementar la función `plot_importance` que se encuentra en la lectura de Bagging y Random Forest.

## Ejercicio 4: Probabilidad de fuga

* El gerente general necesita saber en qué estados hay una mayor probabilidad de fuga de clientes. Para ello, identifique los tres estados con una mayor probabilidad de fuga. 
* Implemente el modelo predictivo con el archivo `churn_test.csv`.
* Recuerde que para obtener la probabilidad de clase, debe utilizar la función `predict_proba` del modelo.